In [14]:
from bs4 import BeautifulSoup
import requests
import numpy as np
import time
import re
import os

In [15]:
import scrapy

ModuleNotFoundError: No module named 'scrapy'

In [2]:
SITENAME = "rottentomatoes"
SITEMAP_SEED = 'https://www.rottentomatoes.com/sitemap_64.xml'
PAGES_SAVE_DIR = "pages-rotten"
TABLES_SAVE_DIR = "tables-rotten"
TEMPLATE_DEEP = 3

In [3]:
r = requests.get(SITEMAP_SEED) #URL della sitemap del sito target
xml = r.text
soup = BeautifulSoup(xml)
sitemapTags = soup.find_all("loc")
pages = np.array([])

for page in sitemapTags: 
    pages  = np.append(pages, page.string)

In [9]:
import pandas as pd
pd.options.display.max_colwidth = 100


df = pd.DataFrame(data=pages)
df = df.loc[df[0].str.contains("https://www\..+\.\w+(/[^/]+){"+str((TEMPLATE_DEEP+1))+"}$" , regex = True)]

df.shape
df

E:\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  


,0
0,https://www.rottentomatoes.com/tv/tarzan_and_jane/s01/e08
1,https://www.rottentomatoes.com/tv/the_break/s01/e01
2,https://www.rottentomatoes.com/tv/the_break/s01/e02
3,https://www.rottentomatoes.com/tv/the_break/s01/e03
4,https://www.rottentomatoes.com/tv/the_break/s01/e04
5,https://www.rottentomatoes.com/tv/the_break/s01/e05
6,https://www.rottentomatoes.com/tv/the_break/s01/e06
7,https://www.rottentomatoes.com/tv/the_break/s01/e07
8,https://www.rottentomatoes.com/tv/the_break/s01/e08
9,https://www.rottentomatoes.com/tv/the_break/s01/e09


In [5]:
page_dir = ".\\"+PAGES_SAVE_DIR+"\\"
if not os.path.exists(page_dir):
    os.makedirs(page_dir)
    
t_page_dir = ".\\"+TABLES_SAVE_DIR+"\\"
if not os.path.exists(t_page_dir):
    os.makedirs(t_page_dir)

In [11]:
index = 0
page_ref_table = []
for page in df[0]:
    for p in range(0,TEMPLATE_DEEP):
        page_template = re.sub("(/[^/]+){"+str(p)+"}$" , "" , page)
        resp = requests.get(page_template)
        html = resp.content.decode("utf-8")
        filepath = page_dir+str(index)+"-"+str(p)+'.html'
        f = open(filepath, 'w' , encoding='utf-8')
        f.write(html)
        f.close
        time.sleep(0.5) #intervallo di tempo per non incorrere in filtri anti DDOS
        page_ref_table.append([page_template , p , filepath])
        print("saved "+filepath)
    index += 1
    print("------------------")

saved .\pages-rotten\0-0.html
saved .\pages-rotten\0-1.html
saved .\pages-rotten\0-2.html
------------------
saved .\pages-rotten\1-0.html
saved .\pages-rotten\1-1.html
saved .\pages-rotten\1-2.html
------------------
saved .\pages-rotten\2-0.html
saved .\pages-rotten\2-1.html
saved .\pages-rotten\2-2.html
------------------
saved .\pages-rotten\3-0.html
saved .\pages-rotten\3-1.html
saved .\pages-rotten\3-2.html
------------------
saved .\pages-rotten\4-0.html
saved .\pages-rotten\4-1.html
saved .\pages-rotten\4-2.html
------------------
saved .\pages-rotten\5-0.html
saved .\pages-rotten\5-1.html
saved .\pages-rotten\5-2.html
------------------
saved .\pages-rotten\6-0.html
saved .\pages-rotten\6-1.html
saved .\pages-rotten\6-2.html
------------------
saved .\pages-rotten\7-0.html
saved .\pages-rotten\7-1.html
saved .\pages-rotten\7-2.html
------------------
saved .\pages-rotten\8-0.html
saved .\pages-rotten\8-1.html
saved .\pages-rotten\8-2.html
------------------
saved .\pages-rotte

saved .\pages-rotten\73-2.html
------------------
saved .\pages-rotten\74-0.html
saved .\pages-rotten\74-1.html
saved .\pages-rotten\74-2.html
------------------
saved .\pages-rotten\75-0.html
saved .\pages-rotten\75-1.html
saved .\pages-rotten\75-2.html
------------------
saved .\pages-rotten\76-0.html
saved .\pages-rotten\76-1.html
saved .\pages-rotten\76-2.html
------------------
saved .\pages-rotten\77-0.html
saved .\pages-rotten\77-1.html
saved .\pages-rotten\77-2.html
------------------
saved .\pages-rotten\78-0.html
saved .\pages-rotten\78-1.html
saved .\pages-rotten\78-2.html
------------------
saved .\pages-rotten\79-0.html
saved .\pages-rotten\79-1.html
saved .\pages-rotten\79-2.html
------------------
saved .\pages-rotten\80-0.html
saved .\pages-rotten\80-1.html
saved .\pages-rotten\80-2.html
------------------
saved .\pages-rotten\81-0.html
saved .\pages-rotten\81-1.html
saved .\pages-rotten\81-2.html
------------------
saved .\pages-rotten\82-0.html
saved .\pages-rotten\82

saved .\pages-rotten\145-2.html
------------------
saved .\pages-rotten\146-0.html
saved .\pages-rotten\146-1.html
saved .\pages-rotten\146-2.html
------------------
saved .\pages-rotten\147-0.html
saved .\pages-rotten\147-1.html
saved .\pages-rotten\147-2.html
------------------
saved .\pages-rotten\148-0.html
saved .\pages-rotten\148-1.html
saved .\pages-rotten\148-2.html
------------------
saved .\pages-rotten\149-0.html
saved .\pages-rotten\149-1.html
saved .\pages-rotten\149-2.html
------------------
saved .\pages-rotten\150-0.html
saved .\pages-rotten\150-1.html
saved .\pages-rotten\150-2.html
------------------
saved .\pages-rotten\151-0.html
saved .\pages-rotten\151-1.html
saved .\pages-rotten\151-2.html
------------------
saved .\pages-rotten\152-0.html
saved .\pages-rotten\152-1.html
saved .\pages-rotten\152-2.html
------------------
saved .\pages-rotten\153-0.html
saved .\pages-rotten\153-1.html
saved .\pages-rotten\153-2.html
------------------
saved .\pages-rotten\154-0.ht

saved .\pages-rotten\217-0.html
saved .\pages-rotten\217-1.html
saved .\pages-rotten\217-2.html
------------------
saved .\pages-rotten\218-0.html
saved .\pages-rotten\218-1.html
saved .\pages-rotten\218-2.html
------------------
saved .\pages-rotten\219-0.html
saved .\pages-rotten\219-1.html
saved .\pages-rotten\219-2.html
------------------
saved .\pages-rotten\220-0.html
saved .\pages-rotten\220-1.html
saved .\pages-rotten\220-2.html
------------------
saved .\pages-rotten\221-0.html
saved .\pages-rotten\221-1.html
saved .\pages-rotten\221-2.html
------------------
saved .\pages-rotten\222-0.html
saved .\pages-rotten\222-1.html
saved .\pages-rotten\222-2.html
------------------
saved .\pages-rotten\223-0.html
saved .\pages-rotten\223-1.html
saved .\pages-rotten\223-2.html
------------------
saved .\pages-rotten\224-0.html
saved .\pages-rotten\224-1.html
saved .\pages-rotten\224-2.html
------------------
saved .\pages-rotten\225-0.html
saved .\pages-rotten\225-1.html
saved .\pages-ro

saved .\pages-rotten\288-1.html
saved .\pages-rotten\288-2.html
------------------
saved .\pages-rotten\289-0.html
saved .\pages-rotten\289-1.html
saved .\pages-rotten\289-2.html
------------------
saved .\pages-rotten\290-0.html
saved .\pages-rotten\290-1.html
saved .\pages-rotten\290-2.html
------------------
saved .\pages-rotten\291-0.html
saved .\pages-rotten\291-1.html
saved .\pages-rotten\291-2.html
------------------
saved .\pages-rotten\292-0.html
saved .\pages-rotten\292-1.html
saved .\pages-rotten\292-2.html
------------------
saved .\pages-rotten\293-0.html
saved .\pages-rotten\293-1.html
saved .\pages-rotten\293-2.html
------------------
saved .\pages-rotten\294-0.html
saved .\pages-rotten\294-1.html
saved .\pages-rotten\294-2.html
------------------
saved .\pages-rotten\295-0.html
saved .\pages-rotten\295-1.html
saved .\pages-rotten\295-2.html
------------------
saved .\pages-rotten\296-0.html
saved .\pages-rotten\296-1.html
saved .\pages-rotten\296-2.html
----------------

...

saved .\pages-rotten\1488-0.html
saved .\pages-rotten\1488-1.html
saved .\pages-rotten\1488-2.html
------------------
saved .\pages-rotten\1489-0.html
saved .\pages-rotten\1489-1.html
saved .\pages-rotten\1489-2.html
------------------
saved .\pages-rotten\1490-0.html
saved .\pages-rotten\1490-1.html
saved .\pages-rotten\1490-2.html
------------------
saved .\pages-rotten\1491-0.html
saved .\pages-rotten\1491-1.html
saved .\pages-rotten\1491-2.html
------------------
saved .\pages-rotten\1492-0.html
saved .\pages-rotten\1492-1.html
saved .\pages-rotten\1492-2.html
------------------
saved .\pages-rotten\1493-0.html
saved .\pages-rotten\1493-1.html
saved .\pages-rotten\1493-2.html
------------------
saved .\pages-rotten\1494-0.html
saved .\pages-rotten\1494-1.html
saved .\pages-rotten\1494-2.html
------------------
saved .\pages-rotten\1495-0.html
saved .\pages-rotten\1495-1.html
saved .\pages-rotten\1495-2.html
------------------
saved .\pages-rotten\1496-0.html
saved .\pages-rotten\14

saved .\pages-rotten\1557-2.html
------------------
saved .\pages-rotten\1558-0.html
saved .\pages-rotten\1558-1.html
saved .\pages-rotten\1558-2.html
------------------
saved .\pages-rotten\1559-0.html
saved .\pages-rotten\1559-1.html
saved .\pages-rotten\1559-2.html
------------------
saved .\pages-rotten\1560-0.html
saved .\pages-rotten\1560-1.html
saved .\pages-rotten\1560-2.html
------------------
saved .\pages-rotten\1561-0.html
saved .\pages-rotten\1561-1.html
saved .\pages-rotten\1561-2.html
------------------
saved .\pages-rotten\1562-0.html
saved .\pages-rotten\1562-1.html
saved .\pages-rotten\1562-2.html
------------------
saved .\pages-rotten\1563-0.html
saved .\pages-rotten\1563-1.html
saved .\pages-rotten\1563-2.html
------------------
saved .\pages-rotten\1564-0.html
saved .\pages-rotten\1564-1.html
saved .\pages-rotten\1564-2.html
------------------
saved .\pages-rotten\1565-0.html
saved .\pages-rotten\1565-1.html
saved .\pages-rotten\1565-2.html
------------------
save

saved .\pages-rotten\1627-0.html
saved .\pages-rotten\1627-1.html
saved .\pages-rotten\1627-2.html
------------------
saved .\pages-rotten\1628-0.html
saved .\pages-rotten\1628-1.html
saved .\pages-rotten\1628-2.html
------------------
saved .\pages-rotten\1629-0.html
saved .\pages-rotten\1629-1.html
saved .\pages-rotten\1629-2.html
------------------
saved .\pages-rotten\1630-0.html
saved .\pages-rotten\1630-1.html
saved .\pages-rotten\1630-2.html
------------------
saved .\pages-rotten\1631-0.html
saved .\pages-rotten\1631-1.html
saved .\pages-rotten\1631-2.html
------------------
saved .\pages-rotten\1632-0.html
saved .\pages-rotten\1632-1.html
saved .\pages-rotten\1632-2.html
------------------
saved .\pages-rotten\1633-0.html
saved .\pages-rotten\1633-1.html
saved .\pages-rotten\1633-2.html
------------------
saved .\pages-rotten\1634-0.html
saved .\pages-rotten\1634-1.html
saved .\pages-rotten\1634-2.html
------------------
saved .\pages-rotten\1635-0.html
saved .\pages-rotten\16

saved .\pages-rotten\1696-2.html
------------------
saved .\pages-rotten\1697-0.html
saved .\pages-rotten\1697-1.html
saved .\pages-rotten\1697-2.html
------------------
saved .\pages-rotten\1698-0.html
saved .\pages-rotten\1698-1.html
saved .\pages-rotten\1698-2.html
------------------
saved .\pages-rotten\1699-0.html
saved .\pages-rotten\1699-1.html
saved .\pages-rotten\1699-2.html
------------------
saved .\pages-rotten\1700-0.html
saved .\pages-rotten\1700-1.html
saved .\pages-rotten\1700-2.html
------------------
saved .\pages-rotten\1701-0.html
saved .\pages-rotten\1701-1.html
saved .\pages-rotten\1701-2.html
------------------
saved .\pages-rotten\1702-0.html
saved .\pages-rotten\1702-1.html
saved .\pages-rotten\1702-2.html
------------------
saved .\pages-rotten\1703-0.html
saved .\pages-rotten\1703-1.html
saved .\pages-rotten\1703-2.html
------------------
saved .\pages-rotten\1704-0.html
saved .\pages-rotten\1704-1.html
saved .\pages-rotten\1704-2.html
------------------
save

saved .\pages-rotten\1766-0.html
saved .\pages-rotten\1766-1.html
saved .\pages-rotten\1766-2.html
------------------
saved .\pages-rotten\1767-0.html
saved .\pages-rotten\1767-1.html
saved .\pages-rotten\1767-2.html
------------------
saved .\pages-rotten\1768-0.html
saved .\pages-rotten\1768-1.html
saved .\pages-rotten\1768-2.html
------------------
saved .\pages-rotten\1769-0.html
saved .\pages-rotten\1769-1.html
saved .\pages-rotten\1769-2.html
------------------
saved .\pages-rotten\1770-0.html
saved .\pages-rotten\1770-1.html
saved .\pages-rotten\1770-2.html
------------------
saved .\pages-rotten\1771-0.html
saved .\pages-rotten\1771-1.html
saved .\pages-rotten\1771-2.html
------------------
saved .\pages-rotten\1772-0.html
saved .\pages-rotten\1772-1.html
saved .\pages-rotten\1772-2.html
------------------
saved .\pages-rotten\1773-0.html
saved .\pages-rotten\1773-1.html
saved .\pages-rotten\1773-2.html
------------------
saved .\pages-rotten\1774-0.html
saved .\pages-rotten\17

In [ ]:
pages_array = np.array(page_ref_table)    
np.save(t_page_dir+"pages.npy" , pages_array)

In [ ]:
a = np.load(t_page_dir+"pages.npy")

In [ ]:
a